## Model2 : Multiple tree based and regression  models are trained and the best one is selected

### Model2 takes into consideration the stock name, associated sentiment, subjectivity of the post, topic and other significant factors for prediction

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("Preprocessed_final_data.csv")

In [4]:
df.head()

,sentiment_score,Open,High,Low,Close,Volume,Dividends,Stock Splits,sentiment_score_average,subjectivity,...,Stock_AAPL,Stock_AMZN,Stock_GOOGL,Stock_MSFT,Stock_TSLA,Year,Month,Day,Weekday,Quarter
0,0.104167,234.809998,237.809998,233.970001,237.330002,28481400.0,0.00,0.0,0.122917,0.225000,...,1,0,0,0,0,2024,11,29,4,4
1,0.141667,234.809998,237.809998,233.970001,237.330002,28481400.0,0.00,0.0,0.122917,0.341667,...,1,0,0,0,0,2024,11,29,4,4
2,0.104167,228.059998,230.720001,228.059998,229.869995,38168300.0,0.00,0.0,0.104167,0.225000,...,1,0,0,0,0,2024,11,22,4,4
3,0.104167,226.399994,226.919998,224.270004,225.000000,47923700.0,0.00,0.0,0.104167,0.225000,...,1,0,0,0,0,2024,11,15,4,4
4,0.104167,227.169998,228.660004,226.410004,226.960007,38328800.0,0.25,0.0,0.113826,0.225000,...,1,0,0,0,0,2024,11,8,4,4


## Model Building and Evaluation

In [5]:
! pip install lightgbm
! pip install catboost


     ---------------------------------------- 1.4/1.4 MB 9.1 MB/s eta 0:00:00
     -------------------------------------- 101.8/101.8 MB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 47.1/47.1 kB 2.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.svm import SVR

X = df.drop(columns=['Close'])
y = df['Close']

# Splitting the data for train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42),
    'CatBoost': CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, random_state=42, silent=True),
    'Support Vector Regression': SVR(kernel='rbf')
}

results = {}

# Training the models
for name, model in models.items():
   
    model.fit(X_train, y_train)
    
    # Predicting
    y_pred = model.predict(X_test)
    
    # Evaluating
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Root Mean Squared Error
    mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
    r2 = r2_score(y_test, y_pred)  # R-squared
    
    results[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R-squared': r2
    }

# Evaluation results for each model
results_df = pd.DataFrame(results).T
print("Model Performance Metrics:")
print(results_df)

# Best model 
best_model_name = results_df['R-squared'].idxmax()
best_model = models[best_model_name]

print(f"\nThe best model is {best_model_name} with an R-squared of {results_df.loc[best_model_name, 'R-squared']:.4f}")


C:\Users\hamsa\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.60866e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\hamsa\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.427e+03, tolerance: 1.043e+03
  model = cd_fast.enet_coordinate_descent(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000581 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3105
[LightGBM] [Info] Number of data points in the train set: 1482, number of used features: 22
[LightGBM] [Info] Start training from score 215.991736
Model Performance Metrics:
                                RMSE        MAE  R-squared
Linear Regression           1.477398   1.008038   0.999716
Ridge Regression            1.477093   1.007836   0.999716
Lasso Regression            2.501874   1.656600   0.999185
Random Forest               1.864957   1.150897   0.999547
Gradient Boosting           1.946034   1.374998   0.999507
LightGBM                    2.220278   1.371501   0.999358
CatBoost                    3.917241   2.772935   0.998002
Support Vector

In [6]:
print(results['Ridge Regression'])

{'RMSE': 1.477092603857243, 'MAE': 1.0078355859931742, 'R-squared': 0.9997158867974513}


# Evaluation metrics
## The Root Mean Squared Error in the model is: 1.477092603857243

### Ridge Regression explains 99.97% of the variance in the data, with only 0.03% error in its predictions.

In [28]:
print(y_pred)

[179.68895393 180.12026157 195.56439726 183.15827764 181.3235002
 196.08431208 183.00828194 191.15341744 210.5342161  179.83857391
 182.60668737 206.11346357 183.78851054 197.83968916 179.95441114
 183.74683265 198.30541249 179.69533452 183.79658922 180.58960689
 181.65073008 180.28991734 179.7128636  179.73428888 180.83123936
 199.09417359 181.35573791 183.63487928 180.5347466  181.20431329
 180.23676467 183.6457847  180.07572611 180.96885217 182.10843492
 179.97735782 182.21044732 182.88269411 186.97313436 179.75891472
 183.23802532 188.34440369 183.59070312 183.86248518 203.68029678
 179.69794782 180.22914261 179.74224925 193.13653959 184.07746406
 180.05996557 202.64850385 201.37860082 181.26769033 181.942375
 182.93096526 199.04028262 184.68070063 180.44554642 179.7145672
 182.79812244 179.80501728 181.07403048 180.06652162 191.386681
 180.45033766 181.08927313 198.84026756 184.27122543 181.61726874
 181.53027864 185.21245297 179.79140693 182.41455534 181.22854801
 209.86636591 18

In [10]:
print(X_test)

      sentiment_score        Open        High         Low        Volume  \
233          0.104167  176.223381  177.515329  175.458139  5.198810e+07   
450          0.600000  391.828589  397.655865  385.862110  4.058650e+07   
1240         0.200000  228.020004  233.970001  225.380005  1.124845e+08   
1693         0.432857  175.054966  176.981416  174.014967  1.952204e+07   
411         -0.129167  419.193835  425.838517  411.523031  3.029640e+07   
...               ...         ...         ...         ...           ...   
471         -1.000000  411.647262  411.935650  398.401702  2.691920e+07   
777         -0.200000  189.289993  190.600006  181.869995  7.043560e+07   
413          0.328125  418.904930  420.180074  410.646368  4.289140e+07   
271          0.081299  184.153812  186.148682  183.617867  5.307930e+07   
184          0.136591  197.054638  198.646829  195.203705  6.683160e+07   

      Dividends  Stock Splits  sentiment_score_average  subjectivity  \
233        0.24           0

In [11]:
print(y_pred.shape)

(371,)


In [17]:
print(y_pred)

[179.68895393 180.12026157 195.56439726 183.15827764 181.3235002
 196.08431208 183.00828194 191.15341744 210.5342161  179.83857391
 182.60668737 206.11346357 183.78851054 197.83968916 179.95441114
 183.74683265 198.30541249 179.69533452 183.79658922 180.58960689
 181.65073008 180.28991734 179.7128636  179.73428888 180.83123936
 199.09417359 181.35573791 183.63487928 180.5347466  181.20431329
 180.23676467 183.6457847  180.07572611 180.96885217 182.10843492
 179.97735782 182.21044732 182.88269411 186.97313436 179.75891472
 183.23802532 188.34440369 183.59070312 183.86248518 203.68029678
 179.69794782 180.22914261 179.74224925 193.13653959 184.07746406
 180.05996557 202.64850385 201.37860082 181.26769033 181.942375
 182.93096526 199.04028262 184.68070063 180.44554642 179.7145672
 182.79812244 179.80501728 181.07403048 180.06652162 191.386681
 180.45033766 181.08927313 198.84026756 184.27122543 181.61726874
 181.53027864 185.21245297 179.79140693 182.41455534 181.22854801
 209.86636591 18

In [18]:
first_30 = y_pred[:30]


In [19]:
print(first_30)

[179.68895393 180.12026157 195.56439726 183.15827764 181.3235002
 196.08431208 183.00828194 191.15341744 210.5342161  179.83857391
 182.60668737 206.11346357 183.78851054 197.83968916 179.95441114
 183.74683265 198.30541249 179.69533452 183.79658922 180.58960689
 181.65073008 180.28991734 179.7128636  179.73428888 180.83123936
 199.09417359 181.35573791 183.63487928 180.5347466  181.20431329]


In [22]:
np.save("first_30_pred_model2.npy",first_30)

In [24]:
actual_stockprice = y_test[:30]

In [26]:
actual_stockprice.shape

(30,)

In [27]:
np.save("actual_stockprice.npy",actual_stockprice)

## Model Comparison  

### Model 1 (ARIMA) successfully captures the historical trend in closing prices but does not account for key factors such as stock name, associated sentiment, post subjectivity, and topic.

### In contrast, Model 2 provides more reliable predictions as it is trained with these attributes that significantly impacts stock prices.  


## Next Step 

### In the next step we  combine the prediction results from both models to leverage their strengths and achieve a more accurate stock price forecast.  
